# TensorFlow实现AlexNet

In [1]:
# -*- coding: utf-8 -*-
# author: JiRuiqing
# email: ruiqing0706@gmail.com

# 导包
from datetime import datetime
import math
import time
import tensorflow as tf

In [2]:
# 设定batch大小和数量
batch_size = 32
num_batches = 100

In [3]:
# print卷积层或池化层输出tensor的尺寸
def print_activations(t):
    print(t.op.name, '', t.get_shape().as_list())

In [4]:
# 
def inference(images):
    parameters = []
    
    # 第一层
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11, 11, 3, 64], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv1)
    # 在第1个卷积层后再添加LRN层和最大池化层。
    lrn1 = tf.nn.lrn(conv1, 4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn1')
    pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')
    # 打印输出结果pool1的结构
    print_activations(pool1)
    
    # 第二层
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5, 5, 64, 192], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[192], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv2)
    # 在第2个卷积层后再添加LRN层和最大池化层。
    lrn2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9, beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')
    # 打印输出结果pool2的结构
    print_activations(pool2)
    
    # 第三层
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 192, 384], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[384], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv3)
    
    # 第四层
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 256], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv4)
        
    # 第五层
    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32, stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv5)
    # 在第5个卷积层后再添加最大池化层。
    pool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool5')
    # 打印输出结果pool2的结构
    print_activations(pool5)
    
    return pool5, parameters

In [5]:
# 评估AlexNet每轮计算时间的函数
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s: step %d, duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
            
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec / batch' % (datetime.now(), info_string, num_batches, mn, sd))

In [6]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(tf.random_normal([batch_size, image_size, image_size, 3], dtype=tf.float32, stddev=1e-1))
        
        pool5, parameters = inference(images)
        
        init = tf.global_variables_initializer()
        
        # 这两个session有什么区别呢？
#         session = tf.InteractiveSession()
        session = tf.Session()
        session.run(init)
        
        time_tensorflow_run(session, pool5, "Forward")
        
        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective, parameters)
        time_tensorflow_run(session, grad, "Forward-backward")
    

In [7]:
run_benchmark()

(u'conv1', '', [32, 56, 56, 64])
(u'pool1', '', [32, 27, 27, 64])
(u'conv2', '', [32, 27, 27, 192])
(u'pool2', '', [32, 13, 13, 192])
(u'conv3', '', [32, 13, 13, 384])
(u'conv4', '', [32, 13, 13, 256])
(u'conv5', '', [32, 13, 13, 256])
(u'pool5', '', [32, 6, 6, 256])
2018-01-19 13:49:21.473037: step 0, duration = 1.187
2018-01-19 13:49:31.324412: step 10, duration = 1.068
2018-01-19 13:49:39.905692: step 20, duration = 0.823
2018-01-19 13:49:48.331390: step 30, duration = 0.815
2018-01-19 13:49:56.598201: step 40, duration = 1.160
2018-01-19 13:50:06.644615: step 50, duration = 0.961
2018-01-19 13:50:15.607577: step 60, duration = 0.740
2018-01-19 13:50:24.335431: step 70, duration = 0.907
2018-01-19 13:50:32.334398: step 80, duration = 0.839
2018-01-19 13:50:40.604938: step 90, duration = 0.745
2018-01-19 13:50:47.665467: Forward across 100 steps, 0.874 +/- 0.145 sec / batch
2018-01-19 13:51:16.219632: step 0, duration = 2.330
2018-01-19 13:51:40.705367: step 10, duration = 2.338
2018

In [8]:
# 训练时需要花费大量时间，做预测的效率还可以。

In [ ]:
(u'conv1', '', [32, 56, 56, 64])
(u'pool1', '', [32, 27, 27, 64])
(u'conv2', '', [32, 27, 27, 192])
(u'pool2', '', [32, 13, 13, 192])
(u'conv3', '', [32, 13, 13, 384])
(u'conv4', '', [32, 13, 13, 256])
(u'conv5', '', [32, 13, 13, 256])
(u'pool5', '', [32, 6, 6, 256])

In [ ]:
(u'conv1_1', '', [32, 56, 56, 64])
(u'conv1_1', '', [32, 56, 56, 64])
(u'pool1', '', [32, 27, 27, 64])
(u'conv2', '', [32, 27, 27, 192])
(u'pool2', '', [32, 13, 13, 192])
(u'conv3', '', [32, 13, 13, 384])
(u'conv4', '', [32, 13, 13, 256])
(u'conv5', '', [32, 13, 13, 256])
(u'pool5', '', [32, 6, 6, 256])